# Estruturas Cripográficas - Criptografia e Segurança da Informação

## TP2 - Exercício 1

#### Enunciado

Estes problemas destinam à iniciação do uso do SageMath  em protótipos de esquemas clássicos de chave pública.

1. Construir uma classe Python que implemente o  EdDSA a partir do [“standard” FIPS186-5](https://csrc.nist.gov/publications/detail/fips/186/5/draft)
    1. A implementação deve conter funções para assinar digitalmente e verificar a assinatura.
    2. A implementação da classe deve usar  uma das “Twisted Edwards Curves” definidas no standard e escolhida  na iniciação da classe: a curva  “edwards25519” ou “edwards448”.

# Versão do SageMath 

`SageMath 9.5` (Confirmar não se vou mudar)
METER PASSOS PARA INSTALAR + SECCÇÃO COM IMPORTS EM BAIXO

#### Classe EdDSA - Capítulo 7 do FIPS186-5

#### Testes